In [1]:
import numpy as np 
import random 
import pandas as pd
import math

from tqdm import tqdm_notebook as tqdm

digits = pd.read_csv('mnist_train.csv', header=None, dtype=float)

random_centers = digits.iloc[np.random.randint(0, len(digits), size=(10)), 1:]
# random_centers = digits.iloc[[88, 205, 390, 254, 222, 437, 431, 436, 352, 285], 1:]

X = digits.iloc[:10000, 1:]

In [24]:
%%time
centers = random_centers.values

for i in tqdm(range(500)):
    distances = [((X.iloc[:, :784] - centers[k, :])**2).mean(axis=1) for k in range(len(centers))]
    distances = np.array(distances)
    X['N'] = [np.argmin(distances[:, i]) for i in range(len(X))]
    centers = X.groupby(by='N').mean().values    

HBox(children=(IntProgress(value=0, max=500), HTML(value=u'')))


CPU times: user 6min 57s, sys: 5min 42s, total: 12min 39s
Wall time: 12min 41s


In [31]:
confusion_matrix = []
for i in range(len(centers)):
    confusion_matrix.append([sum(X[digits[0]==i]['N']==j) for j in range(10)])
confusion_matrix

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


[[2, 0, 66, 1, 784, 1, 44, 50, 39, 14],
 [12, 2, 3, 1051, 0, 2, 54, 2, 1, 0],
 [12, 2, 23, 147, 8, 652, 52, 55, 26, 14],
 [46, 7, 268, 54, 6, 22, 33, 582, 9, 5],
 [248, 193, 0, 14, 0, 0, 123, 1, 14, 387],
 [71, 6, 174, 12, 9, 1, 326, 225, 19, 20],
 [1, 0, 4, 47, 14, 4, 65, 5, 776, 98],
 [358, 514, 0, 50, 1, 3, 35, 2, 0, 107],
 [38, 18, 451, 73, 4, 7, 135, 183, 12, 23],
 [366, 311, 2, 10, 6, 0, 46, 13, 2, 222]]